<a href="https://colab.research.google.com/github/subki72/Customer-Cohort-Retention-Analysis/blob/main/Cohort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv("data.csv",encoding= 'ISO-8859-1')
print(data.head(10))
print(data.shape)

In [ ]:
pd.options.display.float_format = '{:,.0f}'.format
data.describe()

In [ ]:
import dateutil
from datetime import datetime as dt
from pytz import utc

data['datetime'] = data['InvoiceDate'].apply(lambda x: dateutil.parser.parse(x).timestamp())
data['month'] = data['datetime'].apply(lambda x: dt.fromtimestamp(x, utc).month)
data['year'] = data['datetime'].apply(lambda x: dt.fromtimestamp(x, utc).year)

data.head()


In [ ]:
data['cohort'] = data.apply(lambda row: (row['year']*100)+ (row['month']), axis=1)
cohorts = data.groupby('CustomerID')['cohort'].min().reset_index()
cohorts.columns = ['CustomerID', 'first_cohort']
data = data.merge(cohorts, on ='CustomerID', how='left')
data.head()

In [ ]:
headers = data['cohort'].value_counts().reset_index()
headers.columns = ['Cohort', 'Count']
headers.head()
headers = headers.sort_values(['Cohort'])['Cohort'].to_list()
headers

In [ ]:
data.dropna(inplace=True)
data['cohort_distance'] = data.apply(lambda row: (headers.index(row['cohort']) - headers.index(row['first_cohort'])) if (row['first_cohort'] != 0 and row['cohort'] != 0) else np.nan, axis=1)
cohort_pivot = pd.pivot_table(data, index='first_cohort', columns='cohort_distance', values='CustomerID', aggfunc=pd.Series.nunique)
cohort_pivot


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

fig_dims = (14,10)
fig, ax = plt.subplots(figsize=fig_dims)
annot = cohort_pivot.astype(str) + '%'

sns.heatmap(
    cohort_pivot,
    annot=annot,
    fmt="",
    mask=cohort_pivot.isnull(),
    ax=ax,
    square=True,
    linewidth=1,
    cmap=sns.cubehelix_palette(8),
    annot_kws={"size":9, "color":"black"}
)

plt.yticks(rotation=0)
plt.show()

